In [1]:
import pandas as pd

In [2]:
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [6]:
# class 열을 타깃으로 사용하고 나머지 열은 특성 배열에 저장

data = wine[['alcohol','sugar','pH',]].to_numpy()
target = wine['class'].to_numpy()

In [71]:
# train과 test 데이터 세트 만들기

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [14]:
# 나누었던 train에서 validation 나누기

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state = 42)


In [16]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target),dt.score(val_input, val_target))

0.9971133028626413 0.864423076923077


In [24]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
np.mean(scores['test_score'])

{'fit_time': array([0.00477529, 0.00552034, 0.00598359, 0.00598454, 0.00498581]), 'score_time': array([0.00103283, 0.        , 0.0009973 , 0.        , 0.0009985 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


0.855300214703487

In [27]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.8558776930480493


In [28]:
# 훈련세트를 섞은 후 10-폴드 교차 검증 수행

splitter = StratifiedKFold(n_splits = 10, shuffle = True , random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [30]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs= -1)

In [32]:
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [34]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [36]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [38]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [42]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])
best_index

{'min_impurity_decrease': 0.0001}


0

In [44]:
# 좀 더 복잡한 매개변수 

params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5,20,1),
          'min_samples_split':range(2,100,10)}

In [45]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [47]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [54]:
from scipy.stats import uniform, randint
rgen = randint(0,10)
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([114, 107,  97, 112,  91, 101, 109,  94,  79,  96], dtype=int64))

In [56]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.89673749, 0.63451856, 0.82565542, 0.38038289, 0.26987309,
       0.60217665, 0.6120959 , 0.39228208, 0.72468772, 0.84013655])

In [60]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
                   'max_depth' : randint(20,50),
                   'min_samples_split' : randint(2,25),
                   'min_samples_leaf': randint(1,25) }

In [62]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter=100, n_jobs=-1, random_state = 42)

In [63]:
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [65]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [69]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [72]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
